In [ ]:
# to access root modules
if __name__ == "__main__" and __package__ is None:
    from sys import path
    from os.path import dirname as dir
    path.append(dir(path[0])+'/dash_app')
    __package__ = "dash_app"

In [ ]:
import pandas as pd
import numpy as np
import datetime
import dateutil
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from charts import stations_map, map_columns, stations_map_hovertemplate
from settings import *


In [ ]:
data_path = DATA_PATH+'Atmospheric_Domain/2.6SurfaceEarth_RadiationBudget/Figure2.11/'
data_csv = data_path + 'Figure2.11_data.csv'

In [ ]:
"""
Tidy data for charts
"""
columns_dict = {'datetime':'year.1',
               'sum__annual__solar_radiation':'Annual Radiation',
               'moving_average_of_sum__5year__solar_radiation':'5 year moving average',
                  }
xls = pd.ExcelFile(
    data_path+'AnnualSolarRadiationatValentiaObservatory_1955_2019.xlsx')
original_df = pd.read_excel(xls, 'NewDataset', usecols=[7,10,11], skiprows=[0])
df = map_columns(columns_dict, original_df)
df = df[df['datetime'].notna()]
df = df[(df['sum__annual__solar_radiation'].notna()) | (df['moving_average_of_sum__5year__solar_radiation'].notna())]
df['datetime'] = pd.to_datetime(df['datetime'],format='%Y')
df['location'] = 'Valentia'
# df.to_csv(data_csv) #commented out as the file should be kept as is

In [ ]:
df = pd.read_csv(data_csv, index_col=0)
df

In [ ]:
data_path = DATA_PATH+'Atmospheric_Domain/2.6SurfaceEarth_RadiationBudget/Figure2.11/'
data_csv = data_path + 'Figure2.11_data.csv'
df = pd.read_csv(data_csv, index_col=0)
sum_annual_trace = go.Scatter(x=df['datetime'],
                            y=df['sum__annual__solar_radiation'],
                         name='Annual Radiation',
                         mode='markers+lines',
                         marker=dict(color=TIMESERIES_COLOR_2,
                                     size=5,
                                     opacity=0.5),
                         line=dict(color=TIMESERIES_COLOR_2,
                                      width=1),
                         hovertemplate='%{x|%Y}<br>' +
                         '<b>Annual Radiation</b><br>' +
                         'R<sub>s</sub>: %{y:.2f} GJ/m<sup>2</sup><br>' +
                         '<extra></extra>'
                         )
moving_avg_trace = go.Scatter(x=df['datetime'],
                            y=df['moving_average_of_sum__5year__solar_radiation'],
                            name='5 Year Moving Average',
                            mode='lines',  # 'line' is default
                            line_shape='spline',
                            line=dict(color=TIMESERIES_COLOR_1,
                                      width=2),
                            hovertemplate='%{x|%Y}<br>' +
                            '<b>5yr Moving Average</b><br>' +
                            'R<sub>s</sub>: %{y:.2f} GJ/m<sup>2</sup><extra></extra>'
                            )


In [ ]:
figure_2_11 = go.Figure(data=[sum_annual_trace, moving_avg_trace], layout=TIMESERIES_LAYOUT)
figure_2_11.update_layout(
    yaxis=dict(title='Solar Radiation, R<sub>s</sub> (GJ/m<sup>2</sup>)'),
    xaxis=dict(title="Year")
)
figure_2_11

In [ ]:
"""
Map 
"""
data_path = DATA_PATH+'Atmospheric_Domain/2.6SurfaceEarth_RadiationBudget/Map2.6/'
df = pd.read_csv(data_path+'Map2.6_StationTable.txt')

autosol_stations = [
    'Malin_head',
    'Belmullet',
    'Gurteen',
    'Valentia_Observatory',
    'JohnstownII',
    'Dublin_Airport']
autosol_df = df[df['name'].isin(autosol_stations)]
synoptic_df = df[~df['name'].isin(autosol_stations)]
synoptic_trend = go.Scattermapbox(
    name='Standard Synoptic',
    lon=standard_df.Longitude,
    lat=standard_df.Latitude,
    marker=dict(color=STATION_COLORS['Synoptic'],
                size=7),
    hovertemplate=stations_map_hovertemplate(standard_df),
)

autosol_trend = go.Scattermapbox(
    name='AUTOSOL Synoptic',
    lon=autosol_df.Longitude,
    lat=autosol_df.Latitude,
    marker=dict(color='blue',
                size=7),
    hovertemplate=stations_map_hovertemplate(autosol_df),
)

stations_map = go.Figure(
    data=[
        synoptic_trend,
        autosol_trend
    ],
    layout=MAP_LAYOUT)
stations_map.update_layout(
    mapbox=dict(bearing=0,
            center=dict(
                lat=54,
                lon=349
            ),
            pitch=0,
            zoom=4.2)
)
    
stations_map